# Fine-tuning a ChatGPT model for classification

ChatGPT fine-tuning can be used to augment a Large Language Model (LLM) to perform tasks, respond in a specified tone or style, and to reliably reproduce output. In this notebook I experiment with fine-tuning a model to act as a classifer for the MNIST dataset.

To learn more you can visit [OpenAI's guide on fine-tuning](https://platform.openai.com/docs/guides/fine-tuning).

## Load the client and format the data

In [ ]:
from openai import OpenAI

import json

client = OpenAI()
num_train = 500
num_validation = 100

with open("mnist_train.csv", "r") as fin, open("mnist_train.jsonl", "w") as fout:
    count = 0
    header = fin.readline()
    for line in fin:
        if count >= num_train:
            break
        label, image = line.split(",")[0], line.split(",")[1:]
        # each image is 28 x 28 pixels
        image_prompts = [{"role": "user", "content": " ".join(image[i * 28:(i * 28) + 28])} for i in range(0, 28)]
        messages = [{"role": "system", "content": "You are an MNIST classification system."}] +  image_prompts + [{"role": "assistant", "content": label}]
        fout.write(json.dumps({"messages": messages}) + "\n")
        count += 1

with open("mnist_test.csv", "r") as fin, open("mnist_test.jsonl", "w") as fout:
    count = 0
    header = fin.readline()
    for line in fin:
        if count >= num_validation:
            break
        label, image = line.split(",")[0], line.split(",")[1:]
        # each image is 28 x 28 pixels
        image_prompts = [{"role": "user", "content": " ".join(image[i * 28:(i * 28) + 28])} for i in range(0, 28)]
        messages = [{"role": "system", "content": "You are an MNIST classification system."}] +  image_prompts + [{"role": "assistant", "content": label}]
        fout.write(json.dumps({"messages": messages}) + "\n")
        count += 1

## Before fine-tuning

In [ ]:
with open("mnist_test.jsonl", "r") as fin:
    data = json.loads(fin.readline())["messages"]
    messages, label = data[:-1], data[-1]

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages
)

completion.choices[0].message.content

## Load the training data and start a fine-tuning job

In [ ]:
training_data = "mnist_train.jsonl"
validation_data = "mnist_test.jsonl"
suffix = "pat-mnist"
hyperparameters = {
    "n_epochs": 2
}

training_file = client.files.create(
    file=open(training_data, "rb"),
    purpose="fine-tune"
)

validation_file = client.files.create(
    file=open(validation_data, "rb"),
    purpose="fine-tune"
)

client.fine_tuning.jobs.create(
    training_file=training_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix,
    validation_file=validation_file.id,
    #hyperparameters=hyperparameters
)

![A graph within the OpenAI UI that shows real time loss for both the training and validation sets as a function of training examples.](mnist_training.png)

We can watch the progress of our training in the Fine-Tuning tab in the OpenAI UI.

In [ ]:
client.fine_tuning.jobs.list()

In [ ]:
results_out = "fine_tuning_results.txt"

job = client.fine_tuning.jobs.retrieve("ftjob-wK04n1fJ89OEXT2CVKChiD71")
results_data = client.files.content(job.result_files[0]).read()
with open(results_out, "wb") as fout:
    fout.write(results_data)

## After fine-tuning

In [ ]:
with open("mnist_test.jsonl", "r") as fin:
    data = json.loads(fin.readline())["messages"]
    messages, label = data[:-1], data[-1]

completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal:pat-mnist:949vlUXK",
  messages=messages
)

print(label["content"])
completion.choices[0].message.content